In [ ]:
%pip install opencv-python

In [ ]:
from PIL import Image
from src.convert_image_to_ascii_art import convert_image_to_ascii
import cv2

capture = cv2.VideoCapture('C:\\Users\\antonslutsky\\Downloads\\2024_03_29_10_17_20_audio_video.mp4')

frameNr = 0


while (True):
    success, frame = capture.read()
    print(frameNr)
    if success and frameNr == 10:
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        im_pil = Image.fromarray(img)
        ascii_art = convert_image_to_ascii(im_pil)
        print(ascii_art)
        break
    # else:
    #     break
    

    frameNr = frameNr+1
 
capture.release()

In [ ]:
%pip install pyautogui

In [ ]:
# Import necessary modules
import cv2
import numpy as np
import pyautogui

# Initialize video parameters
SCREEN_SIZE = tuple(pyautogui.size())
fourcc = cv2.VideoWriter_fourcc(*"XVID")
fps = 12.0
record_seconds = 10  # Set the desired recording duration

# Create the video writer object
out = cv2.VideoWriter(f"output.avi", fourcc, fps, SCREEN_SIZE)

# Main loop for capturing screenshots and writing frames
for i in range(int(record_seconds * fps)):
    img = pyautogui.screenshot()
    frame = np.array(img)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    out.write(frame)
    cv2.imshow("screenshot", frame)
    if cv2.waitKey(1) == ord("q"):
        break

# Clean up
out.release()
cv2.destroyAllWindows()

In [ ]:
import mouse
import keyboard
from datetime import datetime

mouse_events = []
keyboard_events = []

def keyboard_event_hook(event):
    keyboard_events.append((event.time, event.to_json()))


mouse.hook(mouse_events.append)
keyboard.hook(keyboard_event_hook)
keyboard.start_recording()  

keyboard.wait("a")

mouse.unhook(mouse_events.append)
keyboard.unhook(keyboard_event_hook)
keyboard.stop_recording()


In [ ]:
mouse_events

In [ ]:
keyboard_events

In [ ]:
dir(keyboard.KeyboardEvent)

In [ ]:
!conda env list

In [ ]:
import cv2

def video_to_frames(input_loc, output_loc):
    vidcap = cv2.VideoCapture(input_loc)
    success, image = vidcap.read()
    print(success)
    count = 0

    while success:
        cv2.imwrite(f"{output_loc}/frame{count}.jpg", image)  # Save frame as JPEG file
        success, image = vidcap.read()
        count += 1

# Example usage:
video_to_frames("C:/Users/antonslutsky/Dev/azureml-quickstart/spark_pipeline/src/output2.mp4", 'output_directory')

In [ ]:
!az login --tenant 16b3c013-d300-468d-ac64-7eda0820b6d3

In [ ]:
!az ml component create --file ./jobs/extract_frames_job.yml

In [ ]:
!az ml job create --file ./jobs/extract_frames_job.yml

In [ ]:
!az ml job create --file ./jobs/convert_to_ascii_art_spark_job.yml

In [ ]:
!az ml component create --file ./jobs/convert_to_ascii_art_spark_job.yml

In [74]:
!az ml component create --file ./jobs/generate_finetune_training_data_job.yml

{
  "$schema": "http://azureml/sdk-2-0/SparkJob.json",
  "args": "--frames_ascii_dir ${{inputs.frames_ascii_dir}} --captured_data ${{inputs.captured_data}} --captured_data_training ${{outputs.captured_data_training}}",
  "code": "azureml:/subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourceGroups/antonslutsky-rg/providers/Microsoft.MachineLearningServices/workspaces/gpu-workspace/codes/f7d9f246-9ea8-473d-b038-f426a808b510/versions/1",
  "conf": {
    "spark.driver.cores": 8,
    "spark.driver.memory": "56g",
    "spark.executor.cores": 8,
    "spark.executor.instances": 12,
    "spark.executor.memory": "56g"
  },
  "creation_context": {
    "created_at": "2024-04-02T18:25:05.160635+00:00",
    "created_by": "Anton Slutsky",
    "created_by_type": "User",
    "last_modified_at": "2024-04-02T18:25:05.241871+00:00",
    "last_modified_by": "Anton Slutsky",
    "last_modified_by_type": "User"
  },
  "entry": {
    "file": "generate_finetune_training_data_spark.py"
  },
  "environmen


Uploading src (0.04 MBs): 100%|##########| 35494/35494 [00:00<00:00, 80749.77it/s]




In [ ]:
!az ml job create --file ./jobs/finetune_llm_job.yml

In [73]:
!az ml component create --file ./jobs/data_preprocess_job.yml

{
  "$schema": "https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json",
  "code": "azureml:/subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourceGroups/antonslutsky-rg/providers/Microsoft.MachineLearningServices/workspaces/gpu-workspace/codes/d20f40f0-0ac1-40db-94df-6f2295f1bf44/versions/1",
  "command": "python preprocess.py --task_name TextGeneration --text_key '${{inputs.text_key}}' $[[--ground_truth_key '${{inputs.ground_truth_key}}']] $[[--batch_size '${{inputs.batch_size}}']] $[[--pad_to_max_length '${{inputs.pad_to_max_length}}']] $[[--max_seq_length '${{inputs.max_seq_length}}']] $[[--train_file_path '${{inputs.train_file_path}}']] $[[--validation_file_path '${{inputs.validation_file_path}}']] $[[--test_file_path '${{inputs.test_file_path}}']] $[[--train_mltable_path '${{inputs.train_mltable_path}}']] $[[--validation_mltable_path '${{inputs.validation_mltable_path}}']] $[[--test_mltable_path '${{inputs.test_mltable_path}}']] --model_selector_output '${{